## 栈的实现

栈的抽象数据类型由以下结构和操作定义。如上所述，栈被构造为项的有序集合，其中项被添加和从末端移除的位置称为“顶部”。栈是有序的 LIFO 。栈操作如下。

* Stack() 创建一个空的新栈。 它不需要参数，并返回一个空栈。
* push(item)将一个新项添加到栈的顶部。它需要 item 做参数并不返回任何内容。
* pop() 从栈中删除顶部项。它不需要参数并返回 item 。栈被修改。
* peek() 从栈返回顶部项，但不会删除它。不需要参数。 不修改栈。
* isEmpty() 测试栈是否为空。不需要参数，并返回布尔值。
* size() 返回栈中的 item 数量。不需要参数，并返回一个整数。

In [6]:
class Stack(object):
    def __init__(self,limit = 20):
        self.stack = []
        self.limit = limit

    def push(self,num):
        if len(self.stack) >= self.limit:
            raise IndexError("超出栈的承受范围")
        else:
            self.stack.append(num)

    def pop(self):
        if self.stack is not None:
           return self.stack.pop()
        else:
            raise IndexError("栈里面没有元素")

    def peek(self):
        return self.stack[-1]

    def isEmpty(self):
        return not bool(self.stack)

    def size(self):
        return len(self.stack)


## 简单括号匹配

我们现在把注意力转向使用栈解决真正的计算机问题。你会这么写算术表达式

`(5+6)*(7+8)/(4+3)`

其中括号用于命令操作的执行。你可能也有一些语言的经验，如 Lisp 的构造

```lisp
(defun square(n)
     (* n n))
```

这段代码定义了一个名为 square 的函数，它将返回参数的 n 的平方。 Lisp 使用大量的圆括号是臭名昭著的。

在这两个例子中，括号必须以匹配的方式出现。括号匹配意味着每个开始符号具有相应的结束符号，并且括号能被正确嵌套。考虑下面正确匹配的括号字符串：

```
(()()()())

(((())))

(()((())()))
```

对比那些不匹配的括号：

```
((((((())

()))

(()()(()
```

区分括号是否匹配的能力是识别很多编程语言结构的重要部分。具有挑战的是如何编写一个算法，能够从左到右读取一串符号，并决定符号是否平衡。为了解决这个问题，我们需要做一个重要的观察。从左到右处理符号时，最近开始符号必须与下一个关闭符号相匹配。此外，处理的第一个开始符号必须等待直到其匹配最后一个符号。结束符号以相反的顺序匹配开始符号。他们从内到外匹配。这是一个可以用栈解决问题的线索。


一旦你认为栈是保存括号的恰当的数据结构，算法是很直接的。从空栈开始，从左到右处理括号字符串。如果一个符号是一个开始符号，将其作为一个信号，对应的结束符号稍后会出现。另一方面，如果符号是结束符号，弹出栈，只要弹出栈的开始符号可以匹配每个结束符号，则括号保持匹配状态。如果任何时候栈上没有出现符合开始符号的结束符号，则字符串不匹配。最后，当所有符号都被处理后，栈应该是空的。

In [13]:
def parChecker(symbolString):
    s = Stack(len(symbolString))
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol == "(":
            s.push(symbol)
        else:
            if s.isEmpty():
                balanced = False
            else:
                s.pop()

        index = index + 1

    if balanced and s.isEmpty():
        return True
    else:
        return False

print(parChecker('((()))'))
print(parChecker('(()'))

True
False


## 符号匹配

上面显示的匹配括号问题是许多编程语言都会出现的一般情况的特定情况。匹配和嵌套不同种类的开始和结束符号的情况经常发生。例如，在 Python 中，方括号 `[` 和 `]` 用于列表，花括号 `{` 和 `}` 用于字典。括号 `(` 和 `)` 用于元组和算术表达式。只要每个符号都能保持自己的开始和结束关系，就可以混合符号。符号字符串如

```
{ { ( [ ] [ ] ) } ( ) }

[ [ { { ( ( ) ) } } ] ]

[ ] [ ] [ ] ( ) { }
```

这些被恰当的匹配了，因为不仅每个开始符号都有对应的结束符号，而且符号的类型也匹配。

相反这些字符串没法匹配：

````
( [ ) ]

( ( ( ) ] ) )

[ { ( ) ]
````

上节的简单括号检查程序可以轻松的扩展处理这些新类型的符号。回想一下，每个开始符号被简单的压入栈中，等待匹配的结束符号出现。当出现结束符号时，唯一的区别是我们必须检查确保它正确匹配栈顶部开始符号的类型。如果两个符号不匹配，则字符串不匹配。如果整个字符串都被处理完并且没有什么留在栈中，则字符串匹配。

Python 程序见 ActiveCode 1。唯一的变化是 16 行，我们称之为辅助函数匹配。必须检查栈中每个删除的符号，以查看它是否与当前结束符号匹配。如果不匹配，则布尔变量 balanced 被设置为 False。
   

In [ ]:
def parMatch(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol in "([{":
            s.push(symbol)
        else:
            if s.isEmpty():
                balanced = False
            else:
                top = s.pop()
                if not matches(top,symbol):
                       balanced = False
        index = index + 1
    if balanced and s.isEmpty():
        return True
    else:
        return False

def matches(open,close):
    opens = "([{"
    closers = ")]}"
    return opens.index(open) == closers.index(close)


print(parMatch('{{([][])}()}'))
print(parMatch('[{()]'))

## 十进制转换成二进制

在你学习计算机的过程中，你可能已经接触了二进制。二进制在计算机科学中是很重要的，因为存储在计算机内的所有值都是以 0 和 1 存储的。如果没有能力在二进制数和普通字符串之间转换，我们与计算机之间的交互非常棘手。

整数值是常见的数据项。他们一直用于计算机程序和计算。我们在数学课上学习它们，当然最后用十进制或者基数 10 来表示它们。十进制 $$233_{10}$$ 以及对应的二进制表示 $$11101001_2$$ 分别解释为

$$
2\times10^{2} + 3\times10^{1} + 3\times10^{0}
$$

和

$$
1\times2^{7} + 1\times2^{6} + 1\times2^{5} + 0\times2^{4} + 1\times2^{3} + 0\times2^{2} + 0\times2^{1} + 1\times2^{0}
$$

但是我们如何能够容易地将整数值转换为二进制呢？答案是“除 2”算法，它用栈来跟踪二进制结果的数字。

“除 2” 算法假定我们从大于 0 的整数开始。不断迭代的将十进制除以 2，并跟踪余数。第一个除以 2 的余数说明了这个值是偶数还是奇数。偶数有 0 的余数，记为 0。奇数有余数 1，记为 1.我们将得到的二进制构建为数字序列，第一个余数实际上是序列中的最后一个数字。见 Figure 5 , 我们再次看到了反转的属性，表示栈可能是解决这个问题的数据结构。

![3.8.十进制转换成二进制.figure5](https://raw.githubusercontent.com/facert/python-data-structure-cn/master/3.基本数据结构/3.8.十进制转换成二进制/assets/3.8.十进制转换成二进制.figure5.png)

*Figure 5*

Activecode 1 中的 Python 代码实现了 “除 2” 算法，函数 divideBy2 传入了一个十进制的参数，并重复除以 2。第 7 行使用内置的模运算符 % 来提取余数，第 8 行将余数压到栈上。当除到 0 后，11-13 行构造了一个二进制字符串。

In [14]:
def divideBy2(decNumber):
    remstack = Stack()

    while decNumber > 0:
        rem = decNumber % 2
        remstack.push(rem)
        decNumber = decNumber // 2

    binString = ""
    while not remstack.isEmpty():
        binString = binString + str(remstack.pop())

    return binString

print(divideBy2(42))

101010


这个用于二进制转换的算法可以很容易的扩展以执行任何基数的转换。在计算机科学中，通常会使用很多不同的编码。其中最常见的是二级制，八进制和十六进制。

十进制 $$233$$ 和它对应的八进制和十六进制 $$351_8$$ $$E9_{16}$$

$$ 3\times8^{2} + 5\times8^{1} + 1\times8^{0} $$

和

$$ 14\times16^{1} + 9\times16^{0} $$

可以修改 divideBy2 函数，使它不仅能接受十进制参数，还能接受预期转换的基数。‘除 2’ 的概念被简单的替换成更通用的 ‘除基数’。在 ActiveCode2 展示的是一个名为 baseConverter 函数。采用十进制数和 2 到 16 之间的任何基数作为参数。余数部分仍然入栈，直到被转换的值为 0。我们创建一组数字，用来表示超过 9 的余数。

In [15]:
def baseConverter(decNumber,base):
    digits = "0123456789ABCDEF"

    remstack = Stack()

    while decNumber > 0:
        rem = decNumber % base
        remstack.push(rem)
        decNumber = decNumber // base

    newString = ""
    while not remstack.isEmpty():
        newString = newString + digits[remstack.pop()]

    return newString

print(baseConverter(25,2))
print(baseConverter(25,16))

11001
19
